In social network like Facebook or Twitter, people send friend requests and accept others’ requests as well. Now given two tables as below:


`friend_request`

| sender_id | send_to_id |request_date|
|-----------|------------|------------|
| 1         | 2          | 2016_06-01 |
| 1         | 3          | 2016_06-01 |
| 1         | 4          | 2016_06-01 |
| 2         | 3          | 2016_06-02 |
| 3         | 4          | 2016-06-09 |
 
`request_accepted`

| requester_id | accepter_id |accept_date |
|--------------|-------------|------------|
| 1            | 2           | 2016_06-03 |
| 1            | 3           | 2016-06-08 |
| 2            | 3           | 2016-06-08 |
| 3            | 4           | 2016-06-09 |
| 3            | 4           | 2016-06-10 |


Write a query to find the overall acceptance rate of requests rounded to 2 decimals, which is the number of acceptance divide the number of requests.
 

For the sample data above, your query should return the following result.
 


|accept_rate|
|-----------|
|       0.80|

Note:

The accepted requests are not necessarily from the table friend_request. In this case, you just need to simply count the total accepted requests (no matter whether they are in the original requests), and divide it by the number of requests to get the acceptance rate.

It is possible that a sender sends multiple requests to the same receiver, and a request could be accepted more than once. In this case, the ‘duplicated’ requests or acceptances are only counted once.
If there is no requests at all, you should return 0.00 as the accept_rate.
 

Explanation: There are 4 unique accepted requests, and there are 5 requests in total. So the rate is 0.80.
 

Follow-up:
Can you write a query to return the accept rate but for every month?
How about the cumulative accept rate for every day?


`DataBase Schema:`

------
``` mysql

use leedcode_test;
 
drop table if exists friend_request;
drop table if exists request_accepted;
 
Create table If Not Exists friend_request (
 sender_id INT NOT NULL,
 send_to_id INT NULL,
 request_date DATE NULL);
Create table If Not Exists request_accepted (
 requester_id INT NOT NULL,
 accepter_id INT NULL,
 accept_date DATE NULL);
 
Truncate table friend_request;
insert into friend_request (sender_id, send_to_id, request_date) 
 values ('1', '2', '2016/06/01') , ('1', '3', '2016/06/01') , ('1', '4', '2016/06/01') , ('2', '3', '2016/06/02') , ('3', '4', '2016/06/09');

Truncate table request_accepted;
insert into request_accepted (requester_id, accepter_id, accept_date) 
 values ('1', '2', '2016/06/03'), ('1', '3', '2016/06/08') ,  ('2', '3', '2016/06/08'),  ('3', '4', '2016/06/09') , ('3', '4', '2016/06/10');
```

In [33]:
# Round the number to 2 decimal places:


query('''
SELECT ROUND(135.375, 2);
''')

,"ROUND(135.375, 2)"
0,135.38


In [34]:
# Create connection:
import mysql.connector
import pandas as pd
def query(query):
        mySql_connector = mysql.connector.connect(
        host="localhost",
        user="root",
        passwd="",
        database="leedcode_test")
        
        data= pd.read_sql_query(query , mySql_connector)
        return data
 

In [35]:
query("""SELECT * FROM friend_request;""")

,sender_id,send_to_id,request_date
0,1,2,2016-06-01
1,1,3,2016-06-01
2,1,4,2016-06-01
3,2,3,2016-06-02
4,3,4,2016-06-09


In [36]:
query("""
SELECT * FROM request_accepted;
""")


,requester_id,accepter_id,accept_date
0,1,2,2016-06-03
1,1,3,2016-06-08
2,2,3,2016-06-08
3,3,4,2016-06-09
4,3,4,2016-06-10


In [41]:
query("""
select distinct requester_id, accepter_id from request_accepted;
        """)

,requester_id,accepter_id
0,1,2
1,1,3
2,2,3
3,3,4


In [38]:
query("""
select count(*) from (select distinct requester_id, accepter_id from request_accepted) ;
        """)

,count(*)
0,4


In [39]:
query("""
(select count(*) from (select distinct sender_id, send_to_id from friend_request) as B)
""")

,count(*)
0,5


In [40]:
query('''
select
round(
    ifnull(
    (select count(*) from (select distinct requester_id, accepter_id from request_accepted) as A)
    /
    (select count(*) from (select distinct sender_id, send_to_id from friend_request) as B),
    0)
, 2) as accept_rate;


''')

,accept_rate
0,0.8
